In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import math
import torch
import pandas as pd
import numpy as np
import joblib

In [2]:
uniprot_muta_all = pd.read_csv('./Mutation_Database/MutaDescribe_proteins/uniprot_muta_all.csv')

In [3]:
uniprot_muta_all.head()

,entry,mutant,uniprot_description,rank,wildtype,protein_name,function,GPT_description,all_description,uniprot,mutant_pos,mutation_correct
0,Q8N884-D95A,MQPWHGKAMQRASEAGATAPKASARNARGAPMDPTESPAAPEAALP...,No effect on type I IFN and RSAD2 induction. N...,NaN,MQPWHGKAMQRASEAGATAPKASARNARGAPMDPTESPAAPEAALP...,Cyclic GMP-AMP synthase,Nucleotidyltransferase that catalyzes the form...,NaN,No effect on type I IFN and RSAD2 induction. N...,Q8N884,D95A,True
1,P06276-L153F,MHSKVTIICIRFLFWFLLLCMLIGKSHTEDDIIIATKNGKVRGMNL...,In BCHED; seems to cause reduced expression of...,NaN,MHSKVTIICIRFLFWFLLLCMLIGKSHTEDDIIIATKNGKVRGMNL...,Cholinesterase,Esterase with broad substrate specificity. Con...,The mutation in the BCHE gene leads to reduced...,In BCHED; seems to cause reduced expression of...,P06276,L153F,True
2,P63096-E245L,MGCTLSAEDKAAVERSKMIDRNLREDGEKAAREVKLLLLGAGESGK...,Enhances interaction (inactive GDP-bound) with...,NaN,MGCTLSAEDKAAVERSKMIDRNLREDGEKAAREVKLLLLGAGESGK...,Guanine nucleotide-binding protein G,Guanine nucleotide-binding proteins (G protein...,NaN,Enhances interaction (inactive GDP-bound) with...,P63096,E245L,True
3,O35244-S32A,MPGGLLLGDEAPNFEANTTIGHIRFHDFLGDAWGILFSHPRDFTPV...,Abolishes lipid binding.,NaN,MPGGLLLGDEAPNFEANTTIGHIRFHDFLGDSWGILFSHPRDFTPV...,Peroxiredoxin-6,Thiol-specific peroxidase that catalyzes the r...,"Increased Prdx6 alpha-helical content, key rol...",Abolishes lipid binding. Increased Prdx6 alpha...,O35244,S32A,True
4,P80365-R337C,MERWPWPSGGAWLLVAARALLQLLRSDLRLGRPLLAALALLAALDW...,In AME; decreased half-life from 21 to 4 hours...,NaN,MERWPWPSGGAWLLVAARALLQLLRSDLRLGRPLLAALALLAALDW...,11-beta-hydroxysteroid dehydrogenase type 2,Catalyzes the conversion of biologically activ...,This mutation has been discovered in a consang...,In AME; decreased half-life from 21 to 4 hours...,P80365,R337C,True


In [4]:
uniprot_muta_dict = dict(zip(uniprot_muta_all['mutant'], uniprot_muta_all['uniprot']))
uniprot_wildtype_dict = dict(zip(uniprot_muta_all['wildtype'], uniprot_muta_all['uniprot']))

In [6]:
select_cols = ['BindingDB Reactant_set_id',
 'Ligand SMILES',
 'Ligand InChI',
 'Ligand InChI Key',
 'BindingDB MonomerID',
 'BindingDB Ligand Name',
 'Target Name',
 'Target Source Organism According to Curator or DataSource',
 'Ki (nM)',
 'IC50 (nM)',
 'Kd (nM)',
 'EC50 (nM)',
 'kon (M-1-s-1)',
 'koff (s-1)',
 'pH',
 'PubChem CID',
 'ChEMBL ID of Ligand',
 'DrugBank ID of Ligand',
 'Number of Protein Chains in Target (>1 implies a multichain complex)',
 'BindingDB Target Chain Sequence 1',
 'UniProt (SwissProt) Recommended Name of Target Chain 1',
 'UniProt (SwissProt) Entry Name of Target Chain 1',
 'UniProt (SwissProt) Primary ID of Target Chain 1',
 'UniProt (SwissProt) Secondary ID(s) of Target Chain 1',
 'UniProt (SwissProt) Alternative ID(s) of Target Chain 1']
bindingdb_data = pd.read_csv('./Mutation_Database/BindingDB/BindingDB_All_07282025.tsv', sep='\t', usecols=select_cols)

/var/folders/_0/j8xw5t8j6p3dg7_fq82t36gr0000gp/T/ipykernel_82958/3198557373.py:26: DtypeWarning: Columns (2,3,7,8,9,10,11,12,13,34,35,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  bindingdb_data = pd.read_csv('./Mutation_Database/BindingDB/BindingDB_All_07282025.tsv', sep='\t', usecols=select_cols)


In [7]:
# bindingdb_data_seq = bindingdb_data[bindingdb_data['Target Source Organism According to Curator or DataSource'] == 'Homo sapiens'][ ['BindingDB Target Chain Sequence 1',
#  'UniProt (SwissProt) Recommended Name of Target Chain 1',
#  'UniProt (SwissProt) Entry Name of Target Chain 1',
#  'UniProt (SwissProt) Primary ID of Target Chain 1']].drop_duplicates().reset_index(drop=True)
bindingdb_data_seq = bindingdb_data[ ['BindingDB Target Chain Sequence 1',
 'UniProt (SwissProt) Recommended Name of Target Chain 1',
 'UniProt (SwissProt) Entry Name of Target Chain 1',
 'UniProt (SwissProt) Primary ID of Target Chain 1',
 'Target Source Organism According to Curator or DataSource']].drop_duplicates().reset_index(drop=True)

In [8]:
bindingdb_data_seq.shape

(10319, 5)

In [11]:
for index, row in tqdm(bindingdb_data_seq.iterrows(), total=bindingdb_data_seq.shape[0]):
    if row['BindingDB Target Chain Sequence 1'] in uniprot_muta_dict:
        bindingdb_data_seq.at[index, 'Mutant'] = 'Mutant'
        bindingdb_data_seq.at[index, 'Uniprot'] = uniprot_muta_dict[row['BindingDB Target Chain Sequence 1']]
    elif row['BindingDB Target Chain Sequence 1'] in uniprot_wildtype_dict:
        bindingdb_data_seq.at[index, 'Mutant'] = 'Wildtype'
        bindingdb_data_seq.at[index, 'Uniprot'] = uniprot_wildtype_dict[row['BindingDB Target Chain Sequence 1']]
    else:
        bindingdb_data_seq.at[index, 'Mutant'] = 'No match'
        bindingdb_data_seq.at[index, 'Uniprot'] = 'No match'

100%|██████████| 10319/10319 [00:00<00:00, 42093.44it/s]


In [12]:
bindingdb_data_seq

,BindingDB Target Chain Sequence 1,UniProt (SwissProt) Recommended Name of Target Chain 1,UniProt (SwissProt) Entry Name of Target Chain 1,UniProt (SwissProt) Primary ID of Target Chain 1,Target Source Organism According to Curator or DataSource,Mutant,Uniprot
0,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,Gag-Pol polyprotein,POL_HV1BR,P03367,Human immunodeficiency virus 1,No match,No match
1,PQITLWKRPIVTVKIGGQLREALLDTGADDTVLEDINLPGKWKPKM...,Gag-Pol polyprotein,POL_HV196,Q9QBY3,Human immunodeficiency virus,No match,No match
2,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,Cytochrome P450 3A4,CP3A4_HUMAN,P08684,Homo sapiens,Mutant,P08684
3,MAALRQPQVAELLAEARRAFREEFGAEPELAVSAPGRVNLIGEHTD...,Galactokinase,GALK1_HUMAN,P51570,NaN,Mutant,P51570
4,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,Gag-Pol polyprotein,POL_HV1N5,P12497,Human immunodeficiency virus 1,No match,No match
...,...,...,...,...,...,...,...
10314,MWQEAMRRRRYLRDRSEEAAGGGDGLPRSRDWLYESYYCMSQQHPL...,Adenylate cyclase type 2,ADCY2_HUMAN,Q08462,Homo sapiens,No match,No match
10315,MCVCQTMEVGQYGKNASRAGDRGVLLEPFIHQVGGHSSMMRYDDHT...,Inositol hexakisphosphate kinase 1,IP6K1_HUMAN,Q92551,Homo sapiens,No match,No match
10316,MSPAFRAMDVEPRAKGVLLEPFVHQVGGHSCVLRFNETTLCKPLVP...,Inositol hexakisphosphate kinase 2,IP6K2_HUMAN,Q9UHH9,Homo sapiens,No match,No match
10317,MVVQNSADAGDMRAGVQLEPFLHQVGGHMSVMKYDEHTVCKPLVSR...,Inositol hexakisphosphate kinase 3,IP6K3_HUMAN,Q96PC2,Homo sapiens,Mutant,Q96PC2


In [34]:
bindingdb_data_all = pd.merge(bindingdb_data, bindingdb_data_seq, left_on=list(bindingdb_data_seq)[:-2], right_on=list(bindingdb_data_seq)[:-2], how='inner')

In [13]:
bindingdb_data_mutant = pd.merge(bindingdb_data, bindingdb_data_seq[bindingdb_data_seq['Mutant'] == 'Mutant'], left_on=list(bindingdb_data_seq)[:-2], right_on=list(bindingdb_data_seq)[:-2], how='inner')

In [14]:
bindingdb_data_mutant.head()

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),Kd (nM),EC50 (nM),kon (M-1-s-1),koff (s-1),pH,PubChem CID,ChEMBL ID of Ligand,DrugBank ID of Ligand,Number of Protein Chains in Target (>1 implies a multichain complex),BindingDB Target Chain Sequence 1,UniProt (SwissProt) Recommended Name of Target Chain 1,UniProt (SwissProt) Entry Name of Target Chain 1,UniProt (SwissProt) Primary ID of Target Chain 1,UniProt (SwissProt) Secondary ID(s) of Target Chain 1,UniProt (SwissProt) Alternative ID(s) of Target Chain 1,Mutant,Uniprot
0,143,Cc1nc(CN2CCN(CC2)c2c(Cl)cnc3[nH]c(nc23)-c2cn(C...,InChI=1S/C19H22ClN9O/c1-11-13(9-27(3)25-11)18-...,ZYQKMYRXVHUATB-UHFFFAOYSA-N,247370,"US9447092, 3",Cytochrome P450 3A4,Homo sapiens,NaN,>50000,NaN,NaN,NaN,NaN,NaN,71463198.0,NaN,NaN,1,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,Cytochrome P450 3A4,CP3A4_HUMAN,P08684,P05184 Q16757 Q9UK50,NaN,Mutant,P08684
1,145,O=C1CCCC2=C1C1(CCS(=O)(=O)C1)N=C(Nc1nc3ccccc3o...,InChI=1S/C25H28Cl2O6/c1-6-7-8-9-10-17(15-11-18...,BLBHGUJDTWXHFE-UHFFFAOYSA-N,2764,"2-(benzo[d]oxazol-2-ylamino)-4',5',7,8-tetrahy...",Galactokinase,NaN,NaN,6676.9,NaN,NaN,NaN,NaN,NaN,44640149.0,NaN,NaN,1,MAALRQPQVAELLAEARRAFREEFGAEPELAVSAPGRVNLIGEHTD...,Galactokinase,GALK1_HUMAN,P51570,B2RC07 B4E1G6,NaN,Mutant,P51570
2,199,CN(Cc1ccc(s1)C(=O)N[C@@H](CC(O)=O)C(=O)CSCc1cc...,InChI=1S/C27H27ClN2O7S2/c1-30(12-16-6-8-22(31)...,FIEQQFOHZKVJLV-NRFANRHFSA-N,219,5-({[(5-{[(2S)-1-carboxy-4-{[(2-chlorophenyl)m...,Caspase-3,Homo sapiens,90,NaN,NaN,NaN,NaN,NaN,7.4,5327301.0,NaN,NaN,1,MENTENSVDSKSIKNLEPKIIHGSESMDSGISLDNSYKMDYPEMGL...,Caspase-3,CASP3_HUMAN,P42574,A8K5M2 D3DP53 Q96AN1 Q96KP2,NaN,Mutant,P42574
3,200,OC(=O)C[C@H](NC(=O)c1ccc(CNS(=O)(=O)c2ccc(O)c(...,InChI=1S/C19H18N2O9S/c22-10-13(7-17(24)25)21-1...,FMTZTJFXNZOBIQ-ZDUSSCGKSA-N,220,5-{[(4-{[(2S)-1-carboxy-3-oxopropan-2-yl]carba...,Caspase-1,Homo sapiens,160,NaN,NaN,NaN,NaN,NaN,7.4,5327302.0,NaN,NaN,1,MADKVLKEKRKLFIRSMGEGTINGLLDELLQTRVLNKEEMEKVKRE...,Caspase-1,CASP1_HUMAN,P29466,B5MDZ1 Q53EY6 Q6DMQ1 Q6GSS3 Q6PI75 Q9UCN3,NaN,Mutant,P29466
4,201,OC(=O)C[C@H](NC(=O)c1ccc(CNS(=O)(=O)c2ccc(O)c(...,InChI=1S/C18H17N3O9S/c22-9-12(5-16(24)25)21-17...,BZEURKGKCWHMON-LBPRGKRZSA-N,221,5-{[(5-{[(2S)-1-carboxy-3-oxopropan-2-yl]carba...,Caspase-1,Homo sapiens,3900,NaN,NaN,NaN,NaN,NaN,7.4,5327304.0,NaN,NaN,1,MADKVLKEKRKLFIRSMGEGTINGLLDELLQTRVLNKEEMEKVKRE...,Caspase-1,CASP1_HUMAN,P29466,B5MDZ1 Q53EY6 Q6DMQ1 Q6GSS3 Q6PI75 Q9UCN3,NaN,Mutant,P29466


In [37]:
binding_dti = bindingdb_data_all[~bindingdb_data_all['Kd (nM)'].isna()].reset_index(drop=True)
binding_dti.head()

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),Kd (nM),EC50 (nM),kon (M-1-s-1),koff (s-1),pH,PubChem CID,ChEMBL ID of Ligand,DrugBank ID of Ligand,Number of Protein Chains in Target (>1 implies a multichain complex),BindingDB Target Chain Sequence 1,UniProt (SwissProt) Recommended Name of Target Chain 1,UniProt (SwissProt) Entry Name of Target Chain 1,UniProt (SwissProt) Primary ID of Target Chain 1,UniProt (SwissProt) Secondary ID(s) of Target Chain 1,UniProt (SwissProt) Alternative ID(s) of Target Chain 1,Mutant,Uniprot
0,18379,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,InChI=1S/C13H18N4O3/c1-9(18)6-4-5-7-17-12(19)1...,BYPFEZZEUUWMEJ-UHFFFAOYSA-N,10850,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1,Aspergillus fumigatiaffinis,NaN,NaN,43000.0,NaN,NaN,NaN,NaN,4740.0,CHEMBL628,DB00806,1,MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Endochitinase B1,CHIB1_ASPFM,Q873X9,Q4WB85,NaN,Mutant,Q873X9
1,18380,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,InChI=1S/C13H18N4O3/c1-9(18)6-4-5-7-17-12(19)1...,BYPFEZZEUUWMEJ-UHFFFAOYSA-N,10850,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [A217G],Aspergillus fumigatiaffinis,NaN,NaN,77000.0,NaN,NaN,NaN,NaN,4740.0,CHEMBL628,DB00806,1,MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Endochitinase B1,CHIB1_ASPFM,Q873X9,Q4WB85,NaN,Mutant,Q873X9
2,18381,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,InChI=1S/C13H18N4O3/c1-9(18)6-4-5-7-17-12(19)1...,BYPFEZZEUUWMEJ-UHFFFAOYSA-N,10850,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [D175A],Aspergillus fumigatiaffinis,NaN,NaN,100000.0,NaN,NaN,NaN,NaN,4740.0,CHEMBL628,DB00806,1,MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Endochitinase B1,CHIB1_ASPFM,Q873X9,Q4WB85,NaN,Mutant,Q873X9
3,18382,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,InChI=1S/C13H18N4O3/c1-9(18)6-4-5-7-17-12(19)1...,BYPFEZZEUUWMEJ-UHFFFAOYSA-N,10850,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [D246A],Aspergillus fumigatiaffinis,NaN,NaN,62000.0,NaN,NaN,NaN,NaN,4740.0,CHEMBL628,DB00806,1,MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Endochitinase B1,CHIB1_ASPFM,Q873X9,Q4WB85,NaN,Mutant,Q873X9
4,18383,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,InChI=1S/C13H18N4O3/c1-9(18)6-4-5-7-17-12(19)1...,BYPFEZZEUUWMEJ-UHFFFAOYSA-N,10850,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [E322A],Aspergillus fumigatiaffinis,NaN,NaN,45000.0,NaN,NaN,NaN,NaN,4740.0,CHEMBL628,DB00806,1,MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Endochitinase B1,CHIB1_ASPFM,Q873X9,Q4WB85,NaN,Mutant,Q873X9


In [15]:
mutant_dti = bindingdb_data_mutant[~bindingdb_data_mutant['Kd (nM)'].isna()].reset_index(drop=True)
mutant_dti.head()

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),Kd (nM),EC50 (nM),kon (M-1-s-1),koff (s-1),pH,PubChem CID,ChEMBL ID of Ligand,DrugBank ID of Ligand,Number of Protein Chains in Target (>1 implies a multichain complex),BindingDB Target Chain Sequence 1,UniProt (SwissProt) Recommended Name of Target Chain 1,UniProt (SwissProt) Entry Name of Target Chain 1,UniProt (SwissProt) Primary ID of Target Chain 1,UniProt (SwissProt) Secondary ID(s) of Target Chain 1,UniProt (SwissProt) Alternative ID(s) of Target Chain 1,Mutant,Uniprot
0,18379,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,InChI=1S/C13H18N4O3/c1-9(18)6-4-5-7-17-12(19)1...,BYPFEZZEUUWMEJ-UHFFFAOYSA-N,10850,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1,Aspergillus fumigatiaffinis,NaN,NaN,43000.0,NaN,NaN,NaN,NaN,4740.0,CHEMBL628,DB00806,1,MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Endochitinase B1,CHIB1_ASPFM,Q873X9,Q4WB85,NaN,Mutant,Q873X9
1,18380,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,InChI=1S/C13H18N4O3/c1-9(18)6-4-5-7-17-12(19)1...,BYPFEZZEUUWMEJ-UHFFFAOYSA-N,10850,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [A217G],Aspergillus fumigatiaffinis,NaN,NaN,77000.0,NaN,NaN,NaN,NaN,4740.0,CHEMBL628,DB00806,1,MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Endochitinase B1,CHIB1_ASPFM,Q873X9,Q4WB85,NaN,Mutant,Q873X9
2,18381,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,InChI=1S/C13H18N4O3/c1-9(18)6-4-5-7-17-12(19)1...,BYPFEZZEUUWMEJ-UHFFFAOYSA-N,10850,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [D175A],Aspergillus fumigatiaffinis,NaN,NaN,100000.0,NaN,NaN,NaN,NaN,4740.0,CHEMBL628,DB00806,1,MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Endochitinase B1,CHIB1_ASPFM,Q873X9,Q4WB85,NaN,Mutant,Q873X9
3,18382,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,InChI=1S/C13H18N4O3/c1-9(18)6-4-5-7-17-12(19)1...,BYPFEZZEUUWMEJ-UHFFFAOYSA-N,10850,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [D246A],Aspergillus fumigatiaffinis,NaN,NaN,62000.0,NaN,NaN,NaN,NaN,4740.0,CHEMBL628,DB00806,1,MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Endochitinase B1,CHIB1_ASPFM,Q873X9,Q4WB85,NaN,Mutant,Q873X9
4,18383,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,InChI=1S/C13H18N4O3/c1-9(18)6-4-5-7-17-12(19)1...,BYPFEZZEUUWMEJ-UHFFFAOYSA-N,10850,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [E322A],Aspergillus fumigatiaffinis,NaN,NaN,45000.0,NaN,NaN,NaN,NaN,4740.0,CHEMBL628,DB00806,1,MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Endochitinase B1,CHIB1_ASPFM,Q873X9,Q4WB85,NaN,Mutant,Q873X9


In [16]:
dti_cols = [
 'Ligand SMILES',
 'BindingDB Ligand Name',
 'Target Name',
 'BindingDB Target Chain Sequence 1',
 'Mutant',
 'Uniprot',
 'Kd (nM)']

In [38]:
import numpy as np

mutant_dti_sim = mutant_dti[dti_cols].drop_duplicates()
mutant_dti_sim = mutant_dti_sim[~mutant_dti_sim['Kd (nM)'].isna()].reset_index(drop=True)
import pandas as pd

mutant_dti_sim["Kd (nM)"] = (
    mutant_dti_sim["Kd (nM)"]
    .astype(str)  # 确保是字符串
    .str.replace(r"[<>]", "", regex=True)  # 去掉 > 和 <
    .replace({"N/A": None, "-": None})  # 处理无效值
    .astype(float)
)
mutant_dti_sim['Affinity'] = 9-np.log10(mutant_dti_sim["Kd (nM)"] + 1e-9)


binding_dti_sim = binding_dti[dti_cols].drop_duplicates()
binding_dti_sim = binding_dti_sim[~binding_dti_sim['Kd (nM)'].isna()].reset_index(drop=True)
import pandas as pd
binding_dti_sim["Kd (nM)"] = (
    binding_dti_sim["Kd (nM)"]
    .astype(str)  # 确保是字符串
    .str.replace(r"[<>]", "", regex=True)  # 去掉 > 和 <
    .replace({"N/A": None, "-": None})  # 处理无效值
    .astype(float)
)
binding_dti_sim['Affinity'] = 9-np.log10(binding_dti_sim["Kd (nM)"] + 1e-9)
binding_dti_sim.head()

,Ligand SMILES,BindingDB Ligand Name,Target Name,BindingDB Target Chain Sequence 1,Mutant,Uniprot,Kd (nM),Affinity
0,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1,MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,Q873X9,43000.0,4.366532
1,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [A217G],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,Q873X9,77000.0,4.113509
2,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [D175A],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,Q873X9,100000.0,4.000000
3,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [D246A],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,Q873X9,62000.0,4.207608
4,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [E322A],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,Q873X9,45000.0,4.346787


In [39]:
binding_dti_sim.shape

(97004, 8)

In [18]:
mutant_dti_sim

,Ligand SMILES,BindingDB Ligand Name,Target Name,BindingDB Target Chain Sequence 1,Mutant,Uniprot,Kd (nM),Affinity
0,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1,MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,Q873X9,43000.0,4.366532
1,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [A217G],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,Q873X9,77000.0,4.113509
2,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [D175A],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,Q873X9,100000.0,4.000000
3,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [D246A],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,Q873X9,62000.0,4.207608
4,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [E322A],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,Q873X9,45000.0,4.346787
...,...,...,...,...,...,...,...,...
49793,C[C@@H](O)[C@H](N)C(=O)N[C@@H](CCCCN)C(=O)N[C@...,CHEMBL5398289,Protein AF-9,MASSCAVQVKLELGHRAQVRKKPTVEGFTHDWMVFVRGPEHSNIQH...,Mutant,P42568,48500.0,4.314258
49794,CCN(c1ccc2c(cc(=O)n(CC(C)C)c2c1)C(F)(F)F)c1ccc...,CHEMBL5395307,Retinoic acid receptor RXR-alpha,MDTKHFLPLDFSTQVNSSLTSPTGRGSMAAPSLHPSLGPGIGSPGQ...,Mutant,P19793,6300.0,5.200659
49795,CCN(c1ccc2c(cc(=O)n(CC(C)C)c2c1)C(F)(F)F)c1ccc...,CHEMBL5395307,Retinoic acid receptor RXR-alpha,MDTKHFLPLDFSTQVNSSLTSPTGRGSMAAPSLHPSLGPGIGSPGQ...,Mutant,P19793,10900.0,4.962574
49796,O=C(NC1N=C(c2ccccc2)c2ccccc2NC1=O)c1ccccc1CN1C...,CHEMBL205371,E3 ubiquitin-protein ligase CBL-B,MANSMNGRNPGGRGGNPRKGRILGIIDAIQDAVGPPKQAAADRRTV...,Mutant,Q13191,400.0,6.397940


In [28]:
pan_uniprot_info_target_class = pd.read_csv('/Users/cheng.wang/Documents/Drug-Repo-scRNA/DrugResponseWebExampleData/Webdesign/DrugResponseWebExampleData/DrugActivity/pan_uniprot_info_target_class_chembl_finalized.csv')
pan_uniprot_info_target_class.head()                                           

/var/folders/_0/j8xw5t8j6p3dg7_fq82t36gr0000gp/T/ipykernel_82958/691135947.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  pan_uniprot_info_target_class = pd.read_csv('/Users/cheng.wang/Documents/Drug-Repo-scRNA/DrugResponseWebExampleData/Webdesign/DrugResponseWebExampleData/DrugActivity/pan_uniprot_info_target_class_chembl_finalized.csv')


,Entry,Entry Name,Protein names,Gene Names,Uniprot,Gene,Organism,Length,Sequence,EC number,Active site,Binding site,Rhea ID,Pathway,Target Class,FDA Approved,chembl_component_id,chembl_id,target_pref_name,protein_class_name,Target_Class_Level_Depth,Target_Class_Level_1,Target_Class_Level_2,Target_Class_Level_3,Target_Class_Level_4,Target_Class_Level_5,Target_Class_Level_6
0,A0A0B4J2F0,PIOS1_HUMAN,Protein PIGBOS1 (PIGB opposite strand protein 1),PIGBOS1,A0A0B4J2F0,PIGBOS1,Homo sapiens (Human),54,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLV...,NaN,NaN,NaN,NaN,NaN,Other,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0A0K2S4Q6,CD3CH_HUMAN,Protein CD300H (CD300 antigen-like family memb...,CD300H,A0A0K2S4Q6,CD300H,Homo sapiens (Human),201,MTQRAGAAMLPSALLLLCVPGCLTVSGPSTVMGAVGESLSVQCRYE...,NaN,NaN,NaN,NaN,NaN,Other,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0A0U1RRE5,NBDY_HUMAN,Negative regulator of P-body association (P-bo...,NBDY LINC01420,A0A0U1RRE5,NBDY,Homo sapiens (Human),68,MGDQPCASGRSTLPPGNAREAKPPKKRCLLAPRWDYPEGTPNGGST...,NaN,NaN,NaN,NaN,NaN,Other,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A0A1B0GTW7,CIROP_HUMAN,Ciliated left-right organizer metallopeptidase...,CIROP LMLN2,A0A1B0GTW7,CIROP,Homo sapiens (Human),788,MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKS...,3.4.24.-,"ACT_SITE 306; /evidence=""ECO:0000255|PROSITE-P...","BINDING 305; /ligand=""Zn(2+)""; /ligand_id=""ChE...",NaN,NaN,Enzyme,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A0AV02,S12A8_HUMAN,Solute carrier family 12 member 8 (Cation-chlo...,SLC12A8 CCC9,A0AV02,SLC12A8,Homo sapiens (Human),714,MTQMSQVQELFHEAAQQDALAQPQPWWKTQLFMWEPVLFGTWDGVF...,NaN,NaN,NaN,NaN,NaN,Transporter,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
pan_uniprot_info_target_class_sim = pan_uniprot_info_target_class[['Uniprot','EC number','Target Class', 'FDA Approved','Target_Class_Level_1','Target_Class_Level_2', 'Target_Class_Level_3',
         'Target_Class_Level_4', 'Target_Class_Level_5', 'Target_Class_Level_6']].drop_duplicates().reset_index(drop=True)

In [45]:
binding_dti_sim_class = pd.merge(binding_dti_sim, pan_uniprot_info_target_class_sim, left_on='Uniprot', right_on='Uniprot', how='left').drop(columns=['Uniprot'])
binding_dti_sim_class.head()

,Ligand SMILES,BindingDB Ligand Name,Target Name,BindingDB Target Chain Sequence 1,Mutant,Kd (nM),Affinity,EC number,Target Class,FDA Approved,Target_Class_Level_1,Target_Class_Level_2,Target_Class_Level_3,Target_Class_Level_4,Target_Class_Level_5,Target_Class_Level_6
0,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1,MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,43000.0,4.366532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [A217G],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,77000.0,4.113509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [D175A],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,100000.0,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [D246A],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,62000.0,4.207608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [E322A],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,45000.0,4.346787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
mutant_dti_sim_class = pd.merge(mutant_dti_sim, pan_uniprot_info_target_class_sim, left_on='Uniprot', right_on='Uniprot', how='left').drop(columns=['Uniprot'])
mutant_dti_sim_class.head()

,Ligand SMILES,BindingDB Ligand Name,Target Name,BindingDB Target Chain Sequence 1,Mutant,Kd (nM),Affinity,EC number,Target Class,FDA Approved,Target_Class_Level_1,Target_Class_Level_2,Target_Class_Level_3,Target_Class_Level_4,Target_Class_Level_5,Target_Class_Level_6
0,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1,MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,43000.0,4.366532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [A217G],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,77000.0,4.113509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [D175A],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,100000.0,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [D246A],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,62000.0,4.207608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CC(=O)CCCCn1c(=O)n(C)c2ncn(C)c2c1=O,"1-(5-Oxohexyl)theobromine (pentoxifylline)::3,...",Endochitinase B1 [E322A],MRFATSTIVKVALLLSSLCVDAAVMWNRDTSSTDLEARASSGYRSV...,Mutant,45000.0,4.346787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load ChemRoBERTa tokenizer and model
chem_tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
chem_model = AutoModel.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")

In [48]:
def extract_chem_features(smiles):
    """Extract ChemRoBERTa features from SMILES."""
    try:
        tokens = chem_tokenizer(smiles, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            embeddings = chem_model(**tokens).last_hidden_state.mean(dim=1).squeeze().numpy()
        return embeddings
    except:
        return np.zeros(768)  # Return zero vector if extraction fails

In [49]:
unique_smiles = binding_dti_sim_class['Ligand SMILES'].unique()
chem_features = {smiles: extract_chem_features(smiles) for smiles in tqdm(unique_smiles, desc="Extracting ChemRoBERTa features")}


Extracting ChemRoBERTa features: 100%|██████████| 33739/33739 [07:55<00:00, 70.94it/s]


In [ ]:
# unique_smiles = mutant_dti_sim['Ligand SMILES'].unique()
# chem_features = {smiles: extract_chem_features(smiles) for smiles in tqdm(unique_smiles, desc="Extracting ChemRoBERTa features")}


Extracting ChemRoBERTa features: 100%|██████████| 18842/18842 [04:27<00:00, 70.49it/s]


In [51]:
binding_dti_sim_class['Drug_Features'] = binding_dti_sim_class['Ligand SMILES'].map(chem_features)

In [52]:
mutant_dti_sim_class['Drug_Features'] = mutant_dti_sim_class['Ligand SMILES'].map(chem_features)

In [59]:
# Save datasets
datasets = {'ProteinCLIP_BindingDB':'ProteinCLIP_embedding',
           'MUTAPLM_BindingDB':'MUTAPLM_embedding',
           'ESM2_BindingDB':'ESM2_embedding',
           'ESMv1_BindingDB':'ESMv1_embedding'}
           

In [61]:
def convert_to_float64(arr):
    # Convert list or array to a NumPy array of type float64
    return np.array(arr, dtype=np.float64)

In [63]:
for k,v in datasets.items():
    print(f"Processing dataset: {k} with embedding: {v}")
    dataset = k
    embed = v.split('_')[0]
    protein_embeddings = torch.load(f'/Users/cheng.wang/Documents/Drug-Repo-scRNA/MultiOmicsLLM2024/Embedding_Data/data/BindingDB_embeddings_{embed}.pt',weights_only=False)
    protein_embeddings.head()
    # 确保是纯 dict，不是带 index 的 Series
    protein_embeddings_dict = {
        str(k).strip(): v for k, v in 
        zip(protein_embeddings['BindingDB Target Chain Sequence 1'], 
            protein_embeddings[v])
    }
    binding_dti_sim_class['Target_Features'] = (
    binding_dti_sim_class['BindingDB Target Chain Sequence 1']
    .astype(str).str.strip()
    .map(protein_embeddings_dict)
)
    data = binding_dti_sim_class.copy()
    data['Drug_Features'] = data['Drug_Features'].apply(convert_to_float64)
    data['Target_Features'] = data['Target_Features'].apply(convert_to_float64)
    data.rename(columns={'Ligand SMILES': 'Drug', 'BindingDB Target Chain Sequence 1': 'Target'}, inplace=True)
    from sklearn.model_selection import train_test_split
    # Separate the dataset by unique proteins and drugs
    unique_proteins = data['Target'].unique()
    unique_drugs = data['Drug'].unique()

    # Set the seed for reproducibility
    torch.manual_seed(42)

    # Function to perform a cold split
    def cold_split(unique_items, test_size=0.2, val_size=0.1):
        train_items, test_items = train_test_split(unique_items, test_size=test_size, random_state=42)
        train_items, val_items = train_test_split(train_items, test_size=val_size / (1 - test_size), random_state=42)
        return train_items, val_items, test_items

    # Cold split by proteins
    train_proteins, val_proteins, test_proteins = cold_split(unique_proteins)
    train_cold_protein = data[data['Target'].isin(train_proteins)]
    val_cold_protein = data[data['Target'].isin(val_proteins)]
    test_cold_protein = data[data['Target'].isin(test_proteins)]

    # Cold split by drugs
    train_drugs, val_drugs, test_drugs = cold_split(unique_drugs)
    train_cold_drug = data[data['Drug'].isin(train_drugs)]
    val_cold_drug = data[data['Drug'].isin(val_drugs)]
    test_cold_drug = data[data['Drug'].isin(test_drugs)]

    # Random split of the dataset
    train_random, test_random = train_test_split(data, test_size=0.2, random_state=42)
    train_random, val_random = train_test_split(train_random, test_size=0.1 / 0.8, random_state=42)
    train_cold_protein = train_cold_protein.reset_index(drop=True)
    val_cold_protein = val_cold_protein.reset_index(drop=True)
    test_cold_protein = test_cold_protein.reset_index(drop=True)
    train_cold_drug = train_cold_drug.reset_index(drop=True)
    val_cold_drug = val_cold_drug.reset_index(drop=True)
    test_cold_drug = test_cold_drug.reset_index(drop=True)
    train_random = train_random.reset_index(drop=True)
    val_random = val_random.reset_index(drop=True)
    test_random = test_random.reset_index(drop=True)
    import os
    output_path = "/Users/cheng.wang/Documents/Drug-Repo-scRNA/MultiOmicsLLM2024/BindDBdata/All_BindingDB/{dataset}_{split}/".format(dataset=dataset, split='cold_protein')
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    # Save to Parquet
    train_cold_protein.to_parquet(output_path + "train.parquet")
    val_cold_protein.to_parquet(output_path + "val.parquet")
    test_cold_protein.to_parquet(output_path + "test.parquet")

    output_path = "/Users/cheng.wang/Documents/Drug-Repo-scRNA/MultiOmicsLLM2024/BindDBdata/All_BindingDB/{dataset}_{split}/".format(dataset=dataset, split='cold_drug')
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    # Save to Parquet
    train_cold_drug.to_parquet(output_path + "train.parquet")
    val_cold_drug.to_parquet(output_path + "val.parquet")
    test_cold_drug.to_parquet(output_path + "test.parquet")

    output_path = "/Users/cheng.wang/Documents/Drug-Repo-scRNA/MultiOmicsLLM2024/BindDBdata/All_BindingDB/{dataset}_{split}/".format(dataset=dataset, split='random')
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    # Save to Parquet
    train_random.to_parquet(output_path + "train.parquet")
    val_random.to_parquet(output_path + "val.parquet")
    test_random.to_parquet(output_path + "test.parquet")

Processing dataset: ProteinCLIP_BindingDB with embedding: ProteinCLIP_embedding
Processing dataset: MUTAPLM_BindingDB with embedding: MUTAPLM_embedding
Processing dataset: ESM2_BindingDB with embedding: ESM2_embedding
Processing dataset: ESMv1_BindingDB with embedding: ESMv1_embedding
